In [10]:
# # 仅作为数据处理的测试版本，不作为最终版本
# import nibabel as nib
# import numpy as np
# from PIL import Image

# def nii_to_png(nii_file, output_dir):
#     """
#     将 .nii.gz 文件的每个切片转换为 .png 格式。
    
#     参数：
#     - nii_file: 输入的 .nii.gz 文件路径
#     - output_dir: 输出的保存目录
#     """
#     # 加载NIfTI文件
#     img = nib.load(nii_file)
    
#     # 获取图像数据，通常是一个3D数组（x, y, z）
#     img_data = img.get_fdata()
    
#     # 检查图像数据的维度
#     print(f"图像数据的维度: {img_data.shape}")
    
#     # 遍历每一个切片，通常是 Z 轴方向
#     for slice_idx in range(img_data.shape[2]):  # img_data.shape[2] 是 Z 轴的大小
#         # 选择指定的切片
#         slice_data = img_data[:, :, slice_idx]  # 选择 Z 轴上的切片
        
#         # 将切片数据归一化到 [0, 255] 范围
#         slice_data = np.uint8(np.interp(slice_data, (slice_data.min(), slice_data.max()), (0, 255)))
        
#         # 使用 Pillow 将图像保存为 .png 格式
#         img_pil = Image.fromarray(slice_data)
        
#         # 构建输出文件路径
#         output_file = f"{output_dir}/slice_{slice_idx}.png"
        
#         # 保存为 PNG 文件
#         img_pil.save(output_file)
#         print(f"图像已保存为 {output_file}")

# # 示例使用
# nii_file = 'E:/paperwork/trans_data/ACDC/training/patient001/patient001_frame01.nii.gz'  # 输入的 .nii.gz 文件路径
# output_dir = 'E:/paperwork/trans_data/ACDC/test_output'  # 输出目录，确保该目录已存在

# # 将 .nii.gz 转换为 .png
# nii_to_png(nii_file, output_dir)

图像数据的维度: (216, 256, 10)
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_0.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_1.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_2.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_3.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_4.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_5.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_6.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_7.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_8.png
图像已保存为 E:/paperwork/trans_data/ACDC/test_output/slice_9.png


In [17]:
# 此版为数据处理的最终版本，作用是将ACDC数据集从官网中下载的格式，进行数据格式转换，只需要调整路径即可。
# 将nii.gz格式的数据直接转换成png的格式，对于每个切片都会进行转换，充实了整个数据，可以使数据使用率最大化。
# 其中 4d.nii.gz 格式的文件没有进行转换，直接进行了跳过。
# 生成后图像中，训练集有1902张，测试集有1076z张，其中少部分数据个人认为是脏数据，掩码不规范，即全白，可以进行手动删除。

import nibabel as nib
import numpy as np
from PIL import Image
import os

def nii_to_png(nii_file, output_dir, output_gt_dir):
    """
    将 .nii.gz 文件的每个切片转换为 .png 格式，并分别保存到不同的文件夹。
    
    参数：
    - nii_file: 输入的 .nii.gz 文件路径
    - output_dir: 不包含 '_gt' 的文件的输出保存目录
    - output_gt_dir: 包含 '_gt' 的文件的输出保存目录
    """
    # 加载NIfTI文件
    img = nib.load(nii_file)
    
    # 获取图像数据，通常是一个3D数组（x, y, z）
    img_data = img.get_fdata()
    
    # 检查图像数据的维度
    print(f"图像数据的维度: {img_data.shape}")
    
    # 遍历每一个切片，通常是 Z 轴方向
    for slice_idx in range(img_data.shape[2]):  # img_data.shape[2] 是 Z 轴的大小
        # 选择指定的切片
        slice_data = img_data[:, :, slice_idx]  # 选择 Z 轴上的切片
        
        # 检查切片数据的形状，确保是二维数组
        if slice_data.ndim != 2:
            print(f"切片 {slice_idx} 维度错误: {slice_data.shape}")
            continue  # 跳过这个错误的切片
        
        # 将切片数据归一化到 [0, 255] 范围
        slice_data = np.uint8(np.interp(slice_data, (slice_data.min(), slice_data.max()), (0, 255)))
        
        # 使用 Pillow 将图像保存为 .png 格式
        img_pil = Image.fromarray(slice_data)
        
        # 根据文件名判断输出路径
        if "_gt" in os.path.basename(nii_file):
            output_file = os.path.join(output_gt_dir, os.path.basename(nii_file).replace(".nii.gz", f"_slice_{slice_idx}.png"))
        else:
            output_file = os.path.join(output_dir, os.path.basename(nii_file).replace(".nii.gz", f"_slice_{slice_idx}.png"))
        
        # 保存为 PNG 文件
        img_pil.save(output_file)
        print(f"图像已保存为 {output_file}")

def process_patients(input_dir, output_dir, output_gt_dir):
    """
    遍历指定输入目录下的所有患者文件夹，将每个 .nii.gz 文件转换为 .png。
    
    参数：
    - input_dir: 包含患者文件夹的输入目录
    - output_dir: 不包含 '_gt' 的输出文件夹
    - output_gt_dir: 包含 '_gt' 的输出文件夹
    """
    # 遍历患者文件夹 (patient001, patient002, ..., patient100)
    for patient_num in range(1, 101):
        patient_folder = f"{input_dir}/patient{patient_num:03d}"
        
        # 如果患者文件夹存在，遍历其中的 .nii.gz 文件
        if os.path.exists(patient_folder):
            print(f"处理文件夹: {patient_folder}")
            
            # 遍历文件夹中的每个 .nii.gz 文件
            for filename in os.listdir(patient_folder):
                if filename.endswith(".nii.gz"):
                    nii_file = os.path.join(patient_folder, filename)
                    print(f"处理文件: {nii_file}")
                    
                    # 调用 nii_to_png 函数将 .nii.gz 文件转换为 .png
                    nii_to_png(nii_file, output_dir, output_gt_dir)

# 示例使用
input_dir = 'E:/paperwork/trans_data/ACDC/database/training'  # 输入目录，包含 patient001 到 patient100 文件夹
output_dir = 'E:/paperwork/trans_data/ACDC/ACDC_move/img_dir/train'  # 不包含 '_gt' 的输出目录
output_gt_dir = 'E:/paperwork/trans_data/ACDC/ACDC_move/ann_dir/train'  # 包含 '_gt' 的输出目录

# 创建输出目录（如果不存在）
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_gt_dir, exist_ok=True)

# 处理所有患者文件夹
process_patients(input_dir, output_dir, output_gt_dir)


处理文件夹: E:/paperwork/trans_data/ACDC/database/training/patient001
处理文件: E:/paperwork/trans_data/ACDC/database/training/patient001\patient001_4d.nii.gz
图像数据的维度: (216, 256, 10, 30)
切片 0 维度错误: (216, 256, 30)
切片 1 维度错误: (216, 256, 30)
切片 2 维度错误: (216, 256, 30)
切片 3 维度错误: (216, 256, 30)
切片 4 维度错误: (216, 256, 30)
切片 5 维度错误: (216, 256, 30)
切片 6 维度错误: (216, 256, 30)
切片 7 维度错误: (216, 256, 30)
切片 8 维度错误: (216, 256, 30)
切片 9 维度错误: (216, 256, 30)
处理文件: E:/paperwork/trans_data/ACDC/database/training/patient001\patient001_frame01.nii.gz
图像数据的维度: (216, 256, 10)
图像已保存为 E:/paperwork/trans_data/ACDC/ACDC_move/img_dir/train\patient001_frame01_slice_0.png
图像已保存为 E:/paperwork/trans_data/ACDC/ACDC_move/img_dir/train\patient001_frame01_slice_1.png
图像已保存为 E:/paperwork/trans_data/ACDC/ACDC_move/img_dir/train\patient001_frame01_slice_2.png
图像已保存为 E:/paperwork/trans_data/ACDC/ACDC_move/img_dir/train\patient001_frame01_slice_3.png
图像已保存为 E:/paperwork/trans_data/ACDC/ACDC_move/img_dir/train\patient001_frame01_slice

## 此部分代码仅用于测试，与数据集格式转换无关

In [1]:
import os

import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
label_path = 'E:/paperwork/trans_data/ACDC/ACDC_move/ann_dir/train/patient001_frame01_gt_slice_0.png'
label = cv2.imread(label_path)
label.shape

(216, 256, 3)

In [3]:
np.unique(label)

array([255], dtype=uint8)